In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
all_drives = pd.read_csv('../data/espn_drives2009-2017.csv')

In [3]:
all_drives.head(5)

,Unnamed: 0,away,away_score_after,drive,home,home_score_after,offense,plays,result,time,yds_gained,gameId,uid,TD,FG,punt,turnover,EoH
0,0,PIT,0,0,TEN,0,PIT,3,Punt,1:44,2,290910023,290910023-0,0,0,1,0,0
1,1,PIT,0,1,TEN,0,TEN,3,Punt,1:52,2,290910023,290910023-1,0,0,1,0,0
2,2,PIT,0,2,TEN,0,PIT,5,Punt,3:04,2,290910023,290910023-2,0,0,1,0,0
3,3,PIT,0,3,TEN,0,TEN,6,Missed FG,1:35,38,290910023,290910023-3,0,1,0,0,0
4,4,PIT,0,4,TEN,0,PIT,3,Punt,1:55,-6,290910023,290910023-4,0,0,1,0,0


In [4]:
all_drives['result'].unique()

array(['Punt', 'Missed FG', 'Intercepted Pass', 'Touchdown', 'Fumble',
       'Field Goal', 'End of Half', 'End of Game', 'Downs',
       'End of 1st Half', 'Joshua Cribbs 67 Yard Punt Return',
       'Adrian Peterson 64 Yard Run', 'JAC Timeout', 'NO Timeout',
       'Blocked FG', 'Fumble Touchdown',
       'Desean Jackson 85 Yard Punt Return',
       'Brent Celek 9 Yard Pass From Donovan Mcnabb',
       'Blocked Punt Touchdown', 'Intercepted Pass Touchdown',
       'Julius Jones 62 Yard Run', 'Safety', 'Ryan Grant 3 Yard Run',
       'Blocked Punt', 'Adrian Peterson 27 Yard Run', 'Fumble, Safety',
       'Blocked FG Touchdown',
       'Cadillac Williams 8 Yard Pass From Byron Leftwich', 'BUF Timeout',
       'Frank Gore 79 Yard Run', 'Allen Rossum 67 Yard Punt Return',
       'Will Blackmon 0 Yard Kickoff Return, Tackled In Own End Zone',
       'Percy Harvin 101 Yard Kickoff Return',
       'Ellis Hobbs 0 Yard Kickoff Return, Tackled In Own End Zone',
       'Penalty on Anthony Madis

In [5]:
all_drives['uid'] = all_drives['gameId'].astype(str) +"-"+ all_drives['drive'].astype(str)

In [6]:
all_drives.sample(15)

,Unnamed: 0,away,away_score_after,drive,home,home_score_after,offense,plays,result,time,yds_gained,gameId,uid,TD,FG,punt,turnover,EoH
17314,17314,CHI,14,14,SEA,7,SEA,5,Touchdown,1:58,80,311218003,311218003-14,1,0,0,0,0
34014,34014,GB,21,6,CHI,0,CHI,3,Punt,1:09,1,400554403,400554403-6,0,0,1,0,0
43169,43169,DEN,3,4,IND,3,DEN,5,Punt,2:53,23,400874526,400874526-4,0,0,1,0,0
2067,2067,ATL,0,3,CHI,0,CHI,3,Punt,2:45,9,291018001,291018001-3,0,0,1,0,0
34897,34897,PIT,6,10,NO,14,PIT,7,Missed FG,0:36,44,400554359,400554359-10,0,1,0,0,0
25510,25510,NYJ,27,33,BUF,20,BUF,1,Pass,0:00,0,330922020,330922020-33,0,0,0,0,0
20750,20750,OAK,23,28,JAX,23,JAX,3,Punt,0:30,7,321021013,321021013-28,0,0,1,0,0
25139,25139,OAK,13,15,JAX,3,JAX,3,Punt,1:59,-12,330915013,330915013-15,0,0,1,0,0
51504,51504,CHI,6,15,GB,13,CHI,3,Punt,1:33,2,400951559,400951559-15,0,0,1,0,0
37287,37287,NYG,25,15,WSH,6,NYG,3,Punt,2:08,9,400791484,400791484-15,0,0,1,0,0


In [7]:
# Write a function to count drives in a given dF based on outcome
# Types: TD, FG, Turnover, Punt, End of half/game, Shenanigans
# maybe make columns for each of those types except shenanigans
def find_tds(result_list):
    '''Accepts a list of drive outcomes. 
    Return a binary list of which result in touchdowns'''
    
    is_td = []
    for outcome in [x.lower() for x in result_list]:
        td = 0
        if "touchdown" in outcome:
            if "intercept" in outcome:
                td = 0
            elif "fumble" in outcome:
                td = 0
            elif "punt" in outcome:
                td = 0
            else:
                td = 1
                
        is_td.append(td)
        
    return is_td

def find_fgs(result_list):
    '''Accepts a list of drive outcomes. 
    Return a binary list of which result in field goals'''
    
    is_fg = []
    for outcome in [x.lower() for x in result_list]:
        fg = 0
        if ("field goal" in outcome) or ("fg" in outcome):
            if "block" in outcome:
                fg = 0
            else:
                fg = 1
                
        is_fg.append(fg)
        
    return is_fg

def find_punts(result_list):
    is_punt = []
    for outcome in [x.lower() for x in result_list]:
        punt = 0
        if "punt" in outcome:
            punt = 1
        is_punt.append(punt)
        
    return is_punt
    
def find_turnovers(result_list):
    is_to = []
    for outcome in [x.lower() for x in result_list]:
        to = 0
        if "intercept" in outcome:
            to = 1
        elif "fumble" in outcome:
            to = 1
        elif "downs" in outcome:
            to = 1
        elif "safety" in outcome:
            to = 1
        is_to.append(to)
        
    return is_to
    
def find_endofhalves(result_list):
    is_eoh = []
    for outcome in [x.lower() for x in result_list]:
        eoh = 0
        if "end of half" in outcome:
            eoh = 1
        elif "end of game" in outcome:
            eoh = 1
        is_eoh.append(eoh)
        
    return is_eoh

In [8]:
all_drives['TD'] = find_tds(all_drives['result'].values)

In [9]:
all_drives['FG'] = find_fgs(all_drives['result'].values)
all_drives['punt'] = find_punts(all_drives['result'].values)
all_drives['turnover'] = find_turnovers(all_drives['result'].values)
all_drives['EoH'] = find_endofhalves(all_drives['result'].values)

In [10]:
all_drives.sample(10)

,Unnamed: 0,away,away_score_after,drive,home,home_score_after,offense,plays,result,time,yds_gained,gameId,uid,TD,FG,punt,turnover,EoH
51886,51886,MIA,7,13,TB,20,MIA,1,End of Half,0:45,-4,400981391,400981391-13,0,0,0,0,1
42842,42842,ARI,7,8,NE,10,ARI,3,Punt,0:26,0,400874539,400874539-8,0,0,1,0,0
52150,52150,WSH,0,7,NYG,0,WSH,3,Punt,1:42,8,400951577,400951577-7,0,0,1,0,0
7558,7558,CIN,0,0,TB,0,CIN,6,Punt,3:38,22,301010004,301010004-0,0,0,1,0,0
45906,45906,CAR,13,11,NO,3,CAR,1,Touchdown,0:07,40,400874609,400874609-11,1,0,0,0,0
12317,12317,KC,7,27,BUF,41,KC,3,End of Game,0:59,7,310911012,310911012-27,0,0,0,0,1
10627,10627,BAL,7,8,PIT,0,BAL,6,Punt,4:25,60,301205033,301205033-8,0,0,1,0,0
7299,7299,GB,28,16,DET,20,GB,3,Punt,1:23,-5,301003009,301003009-16,0,0,1,0,0
5013,5013,DET,0,10,ARI,17,DET,3,End of Half,0:11,20,291220008,291220008-10,0,0,0,0,1
29623,29623,NYG,0,3,SEA,3,NYG,5,Punt,2:40,29,331215019,331215019-3,0,0,1,0,0


In [11]:
# Summarize a game's drives
subdf = all_drives[ all_drives['gameId'] == 400874520 ]
# Group by offense, calc what percentage of drives end in TD/FG/etc.
ratios = subdf.groupby(['offense'])[['TD','FG','punt','turnover','EoH']].mean()
np.round(ratios, decimals=3)

,TD,FG,punt,turnover,EoH
offense,,,,,
CLE,0.077,0.077,0.385,0.308,0.154
PHI,0.250,0.250,0.500,0.000,0.000


In [12]:
# drop column Unnamed: 0 and save the dataFrame for future use.
all_drives.drop(['Unnamed: 0'],axis=1,inplace=True)
all_drives.sample(5)

,away,away_score_after,drive,home,home_score_after,offense,plays,result,time,yds_gained,gameId,uid,TD,FG,punt,turnover,EoH
8253,CHI,7,13,WSH,10,WSH,3,Punt,0:51,9,301024003,301024003-13,0,0,1,0,0
14865,DEN,3,5,DET,7,DET,7,Field Goal,2:34,36,311030007,311030007-5,0,1,0,0,0
43750,NE,0,12,BUF,13,BUF,8,Missed FG,3:12,33,400874663,400874663-12,0,1,0,0,0
10597,TB,0,3,ATL,7,TB,11,Touchdown,5:58,86,301205027,301205027-3,1,0,0,0,0
42816,IND,0,4,DET,7,IND,3,Punt,0:38,2,400874573,400874573-4,0,0,1,0,0


In [13]:
all_drives.to_csv('../data/espn_drives2009-2017.csv')

In [14]:
subdf = all_drives[all_drives['offense']=='CAR']
ratios = subdf.groupby(['gameId'])[['TD','FG','punt','turnover','EoH']].mean()
np.round(ratios, decimals=3)

,TD,FG,punt,turnover,EoH
gameId,,,,,
290913029,0.067,0.067,0.267,0.533,0.067
290920001,0.200,0.200,0.200,0.200,0.200
290928006,0.091,0.000,0.545,0.273,0.091
291011029,0.182,0.091,0.273,0.273,0.182
291018027,0.364,0.000,0.273,0.273,0.000
291025029,0.125,0.062,0.500,0.250,0.062
291101022,0.231,0.154,0.462,0.000,0.154
291108018,0.182,0.182,0.273,0.364,0.000
291115029,0.364,0.000,0.364,0.000,0.182


***Get starting field position and time available for the drives***

In [15]:
# Read pbp dataFrame
pbp_df = pd.read_csv('../data/espn_parsedplays2009-2017.csv')
pbp_df.sample(5)

/home/welced12/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (15,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,downdist,detail,play_num,home,away,possession,home_score,away_score,...,week,home_win,is_parseable,is_run,is_pass,is_scramble,is_punt,is_fieldgoal,yds_gained,runpass
241591,241591,4,['1st and 10 at LAR 41'],(13:39 - 4th) T.Austin right end pushed ob at...,5,STL,SF,STL,10,10,...,9,1,True,True,False,False,False,False,3,True
112009,112009,2,['3rd and 6 at BUF 31'],(11:09 - 2nd) R.Fitzpatrick pass incomplete s...,3,BUF,MIA,BUF,3,21,...,11,0,True,False,True,False,False,False,0,True
215710,215710,3,['1st and 10 at LAC 47'],(5:11 - 4th) M.McGloin pass short left to D.M...,4,OAK,SD,OAK,13,26,...,16,0,True,False,True,False,False,False,8,True
117724,117724,8,['1st and 10 at WSH 26'],(5:52 - 1st) S.Greene right tackle to WAS 19 f...,9,NYJ,WSH,NYJ,0,7,...,13,1,False,False,False,False,False,False,x,False
255663,255663,0,[],(10:17 - 4th) M.Bosher kicks 65 yards from ATL...,1,PIT,ATL,PIT,27,20,...,15,1,False,False,False,False,False,False,x,False


In [16]:
pbp_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'downdist', 'detail', 'play_num', 'home',
       'away', 'possession', 'home_score', 'away_score', 'drive_num', 'gameId',
       'down', 'dist', 'home_fieldpos', 'qtr', 'time_rem', 'secs_rem',
       'home_lead', 'total_score', 'adj_lead', 'OT', 'home_possession',
       'season', 'week', 'home_win', 'is_parseable', 'is_run', 'is_pass',
       'is_scramble', 'is_punt', 'is_fieldgoal', 'yds_gained', 'runpass'],
      dtype='object')

In [19]:
# Extract a few select columns for the first play of each drive
relevant_cols = ['detail','drive_num','gameId','secs_rem','home_fieldpos','home_possession']
firstplays = pbp_df.loc[ pbp_df['play_num'] == 1 ][relevant_cols]
firstplays['uid'] = firstplays['gameId'].astype(str)+"-"+firstplays['drive_num'].astype(str)
firstplays[['uid','secs_rem','home_fieldpos','home_possession','detail']].sample(5)

,uid,secs_rem,home_fieldpos,home_possession,detail
136708,320923016-10,1661,23,0,(12:41 - 3rd) A.Peterson right guard to MIN 31...
155929,321118014-11,1857,-15,0,(0:57 - 2nd) N.Folk kicks 63 yards from NYJ 35...
58670,301024001-8,2196,33,0,(6:36 - 2nd) M.Turner left end to ATL 18 for 1...
383071,400951677-1,3464,28,0,(12:44 - 1st) M.Ryan pass incomplete short le...
94582,310926006-8,2540,-30,1,(12:20 - 2nd) R.Grossman pass short middle to ...


In [21]:
# translate the fieldposition to offense's fieldposition
home_fieldpos = firstplays['home_fieldpos'].values
home_poss = firstplays['home_possession'].values
off_fieldpos = [0 for x in home_fieldpos]

for i, tup in enumerate(zip(home_fieldpos, home_poss)):
    (hfp, hp) = tup
    if hp == 1:
        off_fieldpos[i] = hfp
    else:
        off_fieldpos[i] = -1*hfp

firstplays['starting_fieldposition'] = off_fieldpos

In [25]:
# Get ready to merge this information into the drives dataFrame
right_df = firstplays.set_index('uid')[['secs_rem','starting_fieldposition']]

In [28]:
all_drives = pd.merge( all_drives, right_df,
                       how='left',
                       left_on='uid',
                       right_index=True)

In [32]:
all_drives.sample(5)

,away,away_score_after,drive,home,home_score_after,offense,plays,result,time,yds_gained,gameId,uid,TD,FG,punt,turnover,EoH,secs_rem,starting_fieldposition,time_in_secs
52585,TEN,17,17,HOU,13,TEN,3,Punt,1:50,-1,400951728,400951728-17,0,0,1,0,0,277.0,0.0,110
38245,DAL,3,14,NE,13,DAL,1,End of Half,0:03,-1,400791559,400791559-14,0,0,0,0,1,1803.0,0.0,3
40358,GB,0,1,CHI,0,CHI,3,Punt,1:09,7,400791510,400791510-1,0,0,1,0,0,3468.0,-1.0,69
3599,TB,7,16,NO,17,NO,3,Touchdown,1:25,15,291122027,291122027-16,1,0,0,0,0,1621.0,35.0,85
21527,MIN,24,22,DET,17,MIN,2,Touchdown,1:07,75,321111016,321111016-22,1,0,0,0,0,553.0,-25.0,67


In [31]:
# Convert time of possession to seconds
top = all_drives['time'].values
top_secs = [0 for x in top]
for i, time in enumerate(top):
    (mins,secs) = time.split(":")
    total_seconds = int(secs) + 60*int(mins)
    top_secs[i] = total_seconds
    
all_drives['time_in_secs'] = top_secs

all_drives[['time','time_in_secs']].sample(5)

,time,time_in_secs
38769,0:57,57
50127,2:12,132
51529,1:46,106
11471,1:36,96
24855,0:18,18


In [33]:
# Get time remaining in half
rem = all_drives['secs_rem'].values
rem_in_half = [x for x in rem]
for i, time in enumerate(rem):
    if time > 1800:
        rem_in_half[i] -= 1800
        
all_drives['left_in_half'] = rem_in_half

all_drives[['secs_rem','left_in_half']].sample(5)

,secs_rem,left_in_half
39829,3515.0,1715.0
27480,81.0,81.0
50990,3462.0,1662.0
41308,3008.0,1208.0
9985,1801.0,1.0


In [34]:
all_drives.to_csv('../data/espn_drives2009-2017.csv')